In [15]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [16]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [17]:
# Based on the Iterative Policy Evaluation presented in Section 4.1 of Sutton and Barto.
# It based on "in place" updation of the value function
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)        
    while True:             
        delta = 0
        # In a given state s,
        for s in range(env.nS):            
            v = V[s]
            v_temp = 0
            # If agent performs action a
            for a in range(env.nA):
                # Then the environment can blow it in different successor states
                # leading to different reward
                for ns_prob, next_state, reward, done in env.P[s][a]:                    
                    # Update the state-value "in place"
                    v_temp += (policy[s, a] * ns_prob * (reward + discount_factor * V[next_state]))
            
            V[s] = v_temp                        
            delta = max(delta, np.abs(v - V[s]))        
            
        if delta < theta:            
            break
    return np.array(V)

In [18]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:        
        # Evaluate current policy
        V = policy_eval_fn(policy, env, discount_factor)
        # Flag to check optimality of policy
        policy_stable=True
        
        for s in range(env.nS):
            # Select the best action as per the current policy for state s
            old_action = np.argmax(policy[s])
            
            # Calculate the value of each action from the current state,
            # and update policy to choose action with maximum value for
            # current state
            action_value = []
            for a in range(env.nA):
                a_value_temp = 0
                for ns_prob, next_state, reward, done in env.P[s][a]:                                        
                    a_value_temp += (ns_prob * (reward + discount_factor * V[next_state]))
                action_value.append(a_value_temp)            
            # Optimal action in current state
            max_a = np.argmax(action_value)    
            
            # Update policy
            policy[s] = np.eye(env.nA)[max_a]
            
            # Check if the updated policy and old policy are same
            # for the current state. If not, then update flag
            if old_action != max_a:
                policy_stable = False
        
        if policy_stable:
            break
    
    return policy, V

In [19]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [20]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)